In [5]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.2 MB/s eta 0:00:00


In [9]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering
from transformers import BertTokenizerFast

# Load the trained model and tokenizer
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
!ls -l ./saved_distilbert_model


ls: cannot access './saved_distilbert_model': No such file or directory


In [15]:
import os

# Create the directory if it doesn't exist
os.makedirs('./saved_distilbert_model', exist_ok=True)


In [16]:
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering

# Load the model and tokenizer (for example purposes)
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Save the model and tokenizer
model.save_pretrained('./saved_distilbert_model')
tokenizer.save_pretrained('./saved_distilbert_model')


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('./saved_distilbert_model/tokenizer_config.json',
 './saved_distilbert_model/special_tokens_map.json',
 './saved_distilbert_model/vocab.txt',
 './saved_distilbert_model/added_tokens.json',
 './saved_distilbert_model/tokenizer.json')

In [17]:
!ls -l ./saved_distilbert_model


total 260188
-rw-r--r-- 1 root root       561 Jul 29 06:16 config.json
-rw-r--r-- 1 root root 265470032 Jul 29 06:16 model.safetensors
-rw-r--r-- 1 root root       125 Jul 29 06:16 special_tokens_map.json
-rw-r--r-- 1 root root      1196 Jul 29 06:16 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Jul 29 06:16 tokenizer.json
-rw-r--r-- 1 root root    231508 Jul 29 06:16 vocab.txt


In [18]:
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering

# Load the trained model and tokenizer
model = DistilBertForQuestionAnswering.from_pretrained('./saved_distilbert_model')
tokenizer = DistilBertTokenizerFast.from_pretrained('./saved_distilbert_model')


In [20]:
import torch

import ipywidgets as widgets
from IPython.display import display

# Load the trained model and tokenizer
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Create text areas for context and question
context_text = widgets.Textarea(
    description='Context:',
    layout=widgets.Layout(width='100%', height='200px')
)

question_text = widgets.Text(
    description='Question:',
    layout=widgets.Layout(width='100%')
)

# Function to answer the question
def answer_question(button):
    context = context_text.value
    question = question_text.value

    # Tokenize input
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', truncation=True, max_length=512, padding='max_length')

    # Get the start and end scores from the model
    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Get the most likely start and end positions
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    # Convert token indices to text
    input_ids = inputs['input_ids'][0]

    # Handling edge cases where start or end index might be out of bounds
    if start_index < 0 or end_index < 0 or end_index < start_index:
        output_label.value = "Answer: No valid answer found."
        return

    answer_tokens = tokenizer.convert_ids_to_tokens(input_ids[start_index:end_index + 1])
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    # Display the answer
    output_label.value = f"Answer: {answer.strip()}" if answer.strip() else "Answer: No valid answer found."

# Create a button to submit the question
submit_button = widgets.Button(description="Get Answer")
submit_button.on_click(answer_question)

# Create an output widget to display the answer
output_label = widgets.Label(value="")

# Display the widgets
display(context_text, question_text, submit_button, output_label)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Textarea(value='', description='Context:', layout=Layout(height='200px', width='100%'))

Text(value='', description='Question:', layout=Layout(width='100%'))

Button(description='Get Answer', style=ButtonStyle())

Label(value='')